In [54]:
import nibabel as nib
# Load the input NIFTI file and the target NIFTI file
input_nifti = nib.load('/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/atrophy_seeds_2023/shared_analysis/niftis_for_elmira/unsmoothed_atrophy_seeds_v2/sub-0002/ses-01/thresholded_tissue_segment_z_scores/sub-0002_cerebrospinal_fluid.nii')
target_nifti = nib.load('/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/atrophy_seeds_2023/Figures/atrophy_peaks_to_patient_space/atrophy_peak_chance/sphere.nii')
mni_mask_nifti = nib.load('/Users/cu135/hires_backdrops/MNI152_T1_2mm_brain_mask.nii')

Generate Mask

In [20]:
import numpy as np
# Get the data arrays
input_data = input_nifti.get_fdata()
target_data = target_nifti.get_fdata()

# Define the threshold for binarization
threshold = 0  # Replace with your desired threshold value

# Binarize the target data
binary_mask = (target_data > threshold).astype(np.float32)

# Ensure the dimensions of the input data and binary mask match
if input_data.shape != binary_mask.shape:
    raise ValueError("The dimensions of the input NIFTI file and the target NIFTI file do not match.")

# Apply the binary mask to the input data
masked_data = input_data * binary_mask


Optionally Binarize the Masked Data

In [ ]:
masked_data = (masked_data > 0).astype(np.float32)

In [ ]:

# Create a new NIFTI image with the masked data and the original affine
masked_img = nib.Nifti1Image(masked_data, input_nifti.affine, input_nifti.header)

# Save the masked image
nib.save(masked_img, '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/atrophy_seeds_2023/Figures/riMLF_localization/hummingbird_patient_masked_to_rimlf/csf_upsampled_masked_binarized.nii')


Permute Test For Mask Hit

In [55]:
import numpy as np
import random
from tqdm import tqdm

# Get the data arrays
input_data = input_nifti.get_fdata()
target_data = target_nifti.get_fdata()
mni_mask_data = mni_mask_nifti.get_fdata()  # Load the MNI mask

# Ensure the dimensions of the input data, target data, and MNI mask match
if input_data.shape != target_data.shape or input_data.shape != mni_mask_data.shape:
    raise ValueError("The dimensions of the input NIFTI file, the target NIFTI file, and the MNI mask file do not match.")

# Mask the input and target data with the MNI mask
input_data = input_data[mni_mask_data == 1]
target_data = target_data[mni_mask_data == 1]

# Define the threshold for binarization
threshold = 0  # Replace with your desired threshold value

# Binarize the target data
binary_mask = (target_data > threshold).astype(np.float32)
input_data = (input_data > 2).astype(np.float32)

observed = input_data*binary_mask
print(np.sum(observed))

16.0


In [57]:
# Permutation test
hits = 0
num_permutations = 10000

for _ in tqdm(range(num_permutations)):
    # Permute the input data within the MNI mask
    permuted_data = np.random.permutation(input_data)
    
    # Check if permuted data hits the mask
    permuted_masked_data = permuted_data * binary_mask
    if np.sum(permuted_masked_data) > 16:
        hits += 1

print(f"Number of hits: {hits} out of {num_permutations} permutations.")


100%|██████████| 10000/10000 [00:59<00:00, 169.38it/s]

Number of hits: 0 out of 10000 permutations.


In [51]:
print(np.sum(input_data > 0))

14142
